In [ ]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd 
import re
from pecab import PeCab
from collections import defaultdict
pecab = PeCab()


def remove_special_characters(text):
    # 숫자, *, 그리고 큰따옴표(")를 제거하는 정규 표현식 패턴
    pattern = r'[0-9*+."]'
    # 정규 표현식을 사용하여 특수 문자를 제거하고 반환
    return re.sub(pattern, '', text)

# 한국어 텍스트 전처리 분석 함수
def _tokenize(text):
    tokens = []
    for token, pos in pecab.pos(text):
        if pos.startswith('N') :  # 명사 품사만 추출
            tokens.append(token)
    return tokens


def start_():
    model = SentenceTransformer('sentence-transformers/LaBSE') # 임베딩 모델 

    # 텍스트 파일에서 데이터 읽어오기
    with open('tagging_keyword.txt', 'r', encoding='utf-8') as file:
        topic_list = file.readlines()
    # 각 줄의 줄바꿈 문자 제거
    topic_list = [line.strip() for line in topic_list]  # 태깅 키워드 로드 


    with open('tagging_data.txt', 'r', encoding='utf-8') as file:
        data = file.read() 
    # 각 줄을 분리하고 숫자와 마침표를 제거하여 리스트로 변환
    keywords = [line.strip()[:].replace(',', ' ') for line in data.strip().split('\n')]  # 태깅 로드 


    embedded_topic_list = [model.encode(word) for word in topic_list] # 태깅 키워드 임베딩


    return model, topic_list, keywords, embedded_topic_list

def filter_results(results):
        filtered_results = defaultdict(list)

        # 결과를 Chunk_Item_Index로 그룹화
        for hit in results:
            chunk_item_index = hit["_source"]["상품이름"]
            filtered_results[chunk_item_index].append(hit)

        # 각 그룹에서 우선순위를 고려하여 필터링된 결과 반환
        final_results = []
        for chunk_item_index, hits in filtered_results.items():
            # 우선순위를 기준으로 정렬
            hits.sort(key=lambda x: x["_score"], reverse=True)
            # 상위 우선순위 항목 선택
            final_results.append(hits[0])

        return final_results
    


### ElasticSearch 확인

In [ ]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","키바나 비밀번호"),
    ca_certs="http_ca.crt인증서 경로"
)
es.ping()

### 임베딩 모델 로드, 태깅 키워드 로드, 태깅 로드, 태깅 키워드 임베딩

In [ ]:
model, topic_list, keywords, embedded_topic_list = start_()

### 태깅 matching 확인


In [ ]:
user_input = "구성 알찬 부대찌개 추천해줘" # 입력 받기 
user_input2 = _tokenize(user_input) + user_input.split()

user_input2 = user_input2 
input_text = " ".join(user_input2)

# 태깅과 input 텍스트 매칭 
matching_indexes = []
matching_words = []
matches_dict = {}
for word in user_input2:
    for index, keyword in enumerate(keywords):
        if all(sub_word in keyword.split() for sub_word in word.split()):
            matching_indexes.append(index)
            matching_words.append(word)
            matches_dict = {index: word for index, word in zip(matching_indexes, matching_words)}

matching_indexes
user_input2

In [ ]:
user_input3 = []
for keyword in user_input2:
    user_input3.append('*'+keyword+'*')
    
user_input3 = ' '.join(user_input3)
user_input3

### 매칭된 태깅 개수에 따른 쿼리문 작성

In [ ]:

if matching_indexes:
        if len(matching_indexes) >= 2 : 
            selected_embeddings_with_idx = [(embedded_topic_list[idx], idx) for idx in matching_indexes] # 매칭된 태깅의 키워드 임베딩 로드 
            query_em = model.encode(input_text) # 쿼리문 임베딩 
            similarity = [cosine_similarity([query_em], [np.array(embedding)])[0][0] for embedding, idx in selected_embeddings_with_idx] # 태깅의 키워드와 쿼리문 유사도 확인 
            top = max(similarity) # 가장 유사한 값
            top_index_in_similarity = similarity.index(top)  # 유사한 값에 해당하는 인덱스 
            top_index = matching_indexes[top_index_in_similarity]
            word_for_top_index = matches_dict.get(top_index)
        else: 
            top_index = matching_indexes[0]
            word_for_top_index = matching_words[0]
        
        vector_of_input_keyword = model.encode(user_input)

        # 태깅과 상품이름에 가중치를 두어 정렬에 반영 
        query = {
            "query": {
                "bool": {
                    "should": [
                        {
                            "match": {
                                "태깅num": {
                                    "query": top_index,  # 첫 번째 매치된 단어를 사용
                                    "boost": 0.5
                                }
                            }
                        },
                        {
                            "match": {
                                "상품이름": {
                                    "query": word_for_top_index,# 첫 번째 매치된 단어를 사용
                                    "boost": 0.5
                                }
                            }
                        },
                    ]
                }
            },
            "knn": {
                "field": "DescriptionVector",
                "query_vector": vector_of_input_keyword,
                "k": 10,
                "num_candidates": 50,
                "boost": 0.3
            },
            "size": 5,
            "_source": ["상품이름", "가격", "태깅num", "상품정보"]
        }
    # 쿼리 실행
        res = es.search(index="tagging_with", body=query)

    # 필터링된 결과 얻기
        filtered_res = filter_results(res["hits"]["hits"])
    
else:
    print("매칭된 것이 없습니다. ")
    
    vector_of_input_keyword = model.encode(user_input)

    #knn_similarities = cosine_similarity([vector_of_input_keyword], embedded_topic_list)#코사인 계산하여 유사한 토픽 계산
    #top_indices = knn_similarities.argsort()[0][-3:][::-1]
    # 쿼리 실행
    
    query = {
        "query":{
            "query_string":{
                "query": user_input3,
                "default_field": "상품이름"
            }
        },
        "knn":{
            "field": "DescriptionVector",
            "query_vector": vector_of_input_keyword,
            "k": 10,
            "num_candidates": 50,
            "boost" : 0.5
        },
        "size": 5,
        "_source": ["상품이름", "가격", "태깅num", "상품정보"]  
        
    }


    res = es.search(index="tagging_with", body=query)

    # 필터링된 결과 얻기
    filtered_res = filter_results(res["hits"]["hits"])


### 가장 적합한 matching 찾기

In [ ]:
#chatgpt 모델 불러오기 
import openai
import os 
from openai import OpenAI

OPEN_AI_KEY = "openai_key"

openai.api_key = OPEN_AI_KEY

MODEL = 'gpt-3.5-turbo'


MAX_LEN = 1024


client = OpenAI(
    api_key = OPEN_AI_KEY
)


#응답 반환 함수 
def get_openai_response(smry_text):
    
    prompt = '주어진 상품정보를 공백포함 600자로 요약해줘. 요약 시, 상품의 특성 정보가 잘 드러나야 하고 사용자 리뷰가 어떤지도 알려줘야 해. 존댓말 사용해서 친절하게 알려줘 + 상품정보:' + smry_text
    
    completions = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt} 
        
    ]
    
)
    
    return completions.choices[0].message.content
    
    

In [ ]:
i=0
for hit in filtered_res:
    i+=1
    product_name = hit['_source']['상품이름']
    price = hit['_source']['가격']
    product_info = hit['_source']['상품정보']
    product_topic = hit['_source']['태깅num']
    product_info_sum = get_openai_response(product_info)
    formatted_output = f"{i}번째 상품입니다. 토픽 : {product_topic}\n1. 상품: {product_name}\n2. 가격: {price}\n 3.상품정보와 리뷰를 기반으로 한 추천입니다.상품정보와 리뷰를 요약하면 아래와 같습니다:\n{product_info_sum}\n "
    print(formatted_output)